In [7]:
import os
import copy
import torch
from tqdm import tqdm
import pandas as pd
import detectron2
from detectron2.data import detection_utils as utils
from detectron2.utils.logger import setup_logger
setup_logger()

from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

In [8]:
# Register Dataset
try:
    register_coco_instances('coco_trash_test', {}, '/opt/ml/object-detection-level2-cv-04/dataset/test.json', '/opt/ml/object-detection-level2-cv-04/dataset')
except AssertionError:
    pass

In [9]:
# config 불러오기
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file('COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml'))

In [10]:
# config 수정하기
cfg.DATASETS.TEST = ('coco_trash_test',)

cfg.DATALOADER.NUM_WOREKRS = 2

cfg.OUTPUT_DIR = '/opt/ml/object-detection-level2-cv-04/output/10-07_albumtest_allaug_04/train/'

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, 'model_final.pth')

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 10
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.3

In [11]:
# model
predictor = DefaultPredictor(cfg)

# mapper - input data를 어떤 형식으로 return할지
def MyMapper(dataset_dict):
    
    dataset_dict = copy.deepcopy(dataset_dict)
    image = utils.read_image(dataset_dict['file_name'], format='BGR')
    
    dataset_dict['image'] = image
    
    return dataset_dict

# test loader
test_loader = build_detection_test_loader(cfg, 'coco_trash_test', MyMapper)

[10/07 10:37:39 d2.data.datasets.coco]: Loaded 4871 images in COCO format from /opt/ml/object-detection-level2-cv-04/dataset/test.json
[10/07 10:37:39 d2.data.common]: Serializing 4871 elements to byte tensors and concatenating them all ...
[10/07 10:37:39 d2.data.common]: Serialized dataset takes 0.66 MiB


In [12]:
# output 뽑은 후 sumbmission 양식에 맞게 후처리 
prediction_strings = []
file_names = []

class_num = 10

for data in tqdm(test_loader):
    
    prediction_string = ''
    
    data = data[0]
    
    outputs = predictor(data['image'])['instances']
    
    targets = outputs.pred_classes.cpu().tolist()
    boxes = [i.cpu().detach().numpy() for i in outputs.pred_boxes]
    scores = outputs.scores.cpu().tolist()
    
    for target, box, score in zip(targets,boxes,scores):
        prediction_string += (str(target) + ' ' + str(score) + ' ' + str(box[0]) + ' ' 
        + str(box[1]) + ' ' + str(box[2]) + ' ' + str(box[3]) + ' ')
        
    prediction_strings.append(prediction_string)
    file_names.append(data['file_name'].replace('/opt/ml/object-detection-level2-cv-04/dataset/',''))

submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.OUTPUT_DIR, f'submission_det2.csv'), index=None)
submission.head()

100%|██████████| 4871/4871 [08:15<00:00,  9.84it/s]


,PredictionString,image_id
0,7 0.9971253275871277 602.96936 517.41986 955.0...,test/0000.jpg
1,5 0.9100302457809448 132.03397 0.0 496.0875 27...,test/0001.jpg
2,1 0.9843340516090393 427.01743 268.69733 639.6...,test/0002.jpg
3,9 0.9995301961898804 139.11203 260.97153 902.4...,test/0003.jpg
4,9 0.7355118989944458 433.80344 409.58847 653.3...,test/0004.jpg
